In [3]:
from azure.eventhub import EventData
from azure.eventhub.aio import EventHubProducerClient

from open_sky import retrieve_data

from datetime import datetime
import time
import asyncio
import json
import sys

In [5]:
#EVENTHUB Should be the key of the EVENT RESSOURCE
EVENT_HUB_CONNECTION_STR = "Endpoint=sb://opensky.servicebus.windows.net/;SharedAccessKeyName=pyproducer;SharedAccessKeyName=pyconsummer"
EVENT_HUB_NAME = "opensky"

In [6]:
async def send_data(waiting_time):
    #create a producer to send message to the event hub
    #specify a connection string to your event hubs namespace and
    #the event hub name
    producer = EventHubProducerClient.from_connection_string(
        conn_str=EVENT_HUB_CONNECTION_STR,eventhub_name=EVENT_HUB_NAME
    )
    async with producer:
        start = time.time()
        # Add events to the batch.
        while time.time() <= start + waiting_time * 60:
            #create a batch
            event_data_batch = await producer.create_batch()